In [6]:
#必要なものをインストール
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

データセットをつくる
ホールドアウト法で大体の結果を確かめるためのテスト用

In [7]:
df_test=pd.read_csv("df_yy_all_log2.csv")#ここに入れるデータ名を入れ替えていろいろな例を試す
X = df_test.drop('y',axis=1).values
y=df_test.loc[:,["y"]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
y_train= y_train.reshape(-1)
y_test= y_test.reshape(-1)

データセット　本番用


In [12]:
df=pd.read_csv("df_yy_all_log2.csv")
df_t=pd.read_csv('df_yy_all_log_test.csv')
X_train= df.drop('y',axis=1).values
y_train=df.loc[:,["y"]].values
X_test=df_t.values
y_train= y_train.reshape(-1)

学習モデルの作成

In [9]:
rfc=RandomForestClassifier(random_state=0, n_estimators=1000,max_depth= 15,
#                            class_weight="balanced"　:うまくy＝1が予測されるようにしようとするときに試した
                          )
gbct = GradientBoostingClassifier(n_estimators=5000,random_state=0, max_depth=15, learning_rate=0.01)
ada=AdaBoostClassifier(n_estimators=500, learning_rate=0.01, random_state=0)
ext=ExtraTreesClassifier(n_estimators=10000, max_depth =6)
rbf_svm = SVC(kernel='rbf', gamma=0.1, C=1,probability=True)
lgb = lgb.LGBMClassifier(num_leaves = 30,learning_rate=0.1,min_child_samples=20,n_estimators=2000,
#                          class_weight="balanced" :うまくy＝1が予測されるようにしようとするときに試した
                        )
xgb = xgb.XGBClassifier(learning_rate = 0.02,n_estimators= 2000,max_depth= 15,min_child_weight= 1, gamma=0.9,
#                         class_weight="balanced"
                       )

K個に分割してtrainデータの予測値を求める

In [15]:
def learn_model(model):
#yのクラスの比率が等しくなるようにホールドする
#     kf = KFold(n_splits=5, random_state=0, shuffle=True) :通常の分割方法
# 分けたときのyのバランスが良くなるようにしたもの    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    ntrain=X_train.shape[0]
    ntest=X_test.shape[0]
#結果を記録していくためのものをつくる   
    mo_train=np.zeros((ntrain,))
    mo_test = np.zeros((ntest,))
    mo_test_m = np.empty((5, ntest))
    mo_train_pro=np.zeros((ntrain,))
    mo_test_pro = np.zeros((ntest,))
    mo_test_m_pro = np.empty((5, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(X_train,y_train)):
    #分けたデータの1つは予測するためのテストデータとして扱い、残りのデータを使って学習する
        
        X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
        y_train_1, y_test_1 = y_train[train_index], y_train[test_index]

        model.fit(X_train_1,y_train_1)
        print('train accuracy: %.3f' % model.score(X_train_1, y_train_1))
        print('test accuracy: %.3f' % model.score(X_test_1, y_test_1))
        
       
        # 再現率を出力
       
        print(metrics.recall_score(y_test_1, model.predict(X_test_1)))
        # 適合率を出力
        print(metrics.precision_score(y_test_1, model.predict(X_test_1)))
        # F値を出力
        print(metrics.f1_score(y_test_1, model.predict(X_test_1)))
         #クラスの予測や予測確率(_pro)を求める
        mo_train[test_index] =model.predict(X_test_1)
        mo_test_m[i, :] = model.predict(X_test)
        mo_train_pro[test_index] =model.predict_proba(X_test_1)[:,1]
        mo_test_m_pro[i, :] = model.predict_proba(X_test)[:,1]
    #テストデータは5回作られることになるので平均を取る
    mo_test[:]=mo_test_m.mean(axis=0)
    mo_test_pro[:]=mo_test_m_pro.mean(axis=0)
   
    return mo_train.reshape(-1, 1), mo_test.reshape(-1, 1),mo_train_pro.reshape(-1, 1), mo_test_pro.reshape(-1, 1)

学習させるモデルを順番に学習して、train,testの結果を受け取る

In [142]:
#Gridsearchにより最適なパラメータを求める（RandomForest）

# params = {'n_estimators'  : [ 10, 100, 1000, 10000] }
# kf_1 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# gs_rf = GridSearchCV(RandomForestClassifier(), params, cv = kf_1)
# gs_rf.fit(X_train, y_train)
# print(gs_rf.score(X_train,y_train))
# print(gs_rf.best_params_)

1.0
{'n_estimators': 1000}


In [16]:
#RandomForestの学習
print("rfc")
rfc_train,rfc_test,rfc_train_pro,rfc_test_pro=learn_model(rfc)

rfc
train accuracy: 0.968
test accuracy: 0.905
0.352755905511811
0.676737160120846
0.46376811594202894
train accuracy: 0.969
test accuracy: 0.900
0.33070866141732286
0.6461538461538462
0.43750000000000006
train accuracy: 0.969
test accuracy: 0.903
0.3590551181102362
0.6570605187319885
0.46435845213849286
train accuracy: 0.968
test accuracy: 0.905
0.34960629921259845
0.6873065015479877
0.46346555323590816
train accuracy: 0.969
test accuracy: 0.902
0.30757097791798105
0.6794425087108014
0.42345276872964166


In [146]:
#Gridsearch:GrasientBoost

# params = {'n_estimators'  : [ 5000,8000], 
#          'learning_rate' :[0.01,0.005]}
# gs_gb = GridSearchCV(GradientBoostingClassifier(), params, cv = kf_1)
# gs_gb.fit(X_train, y_train)
# print(gs_gb.score(X_train,y_train))

0.9329997699562917


In [145]:
# print(gs_gb.best_params_)

{'learning_rate': 0.01, 'n_estimators': 5000}


In [17]:
#GrasientBoostの学習
print("gbct")
gbct_train,gbct_test,gbct_train_pro,gbct_test_pro=learn_model(gbct)

gbct
train accuracy: 1.000
test accuracy: 0.898
0.44251968503937006
0.5841995841995842
0.503584229390681
train accuracy: 1.000
test accuracy: 0.899
0.431496062992126
0.591792656587473
0.49908925318761393
train accuracy: 1.000
test accuracy: 0.898
0.4393700787401575
0.5873684210526315
0.5027027027027027
train accuracy: 1.000
test accuracy: 0.907
0.47244094488188976
0.6396588486140725
0.5434782608695652
train accuracy: 1.000
test accuracy: 0.901
0.41798107255520506
0.6091954022988506
0.4957904583723106


In [ ]:
#Gridsearch:

# params = {'n_estimators'  : [ 10, 100, 1000, 10000], 
#          'learning_rate' :[0.01,,1.0]}
# gs_ad = GridSearchCV(AdaBoostClassifier(), params, cv = kf_1)
# gs_ad.fit(X_train, y_train)
# print(gs_ad.score(X_train,y_train))

In [18]:
#AdaBoostの学習
print("ad")
ada_train, ada_test,ada_train_pro, ada_test_pro=learn_model(ada)


ad
train accuracy: 0.889
test accuracy: 0.889
0.07401574803149606
0.7580645161290323
0.13486370157819225
train accuracy: 0.889
test accuracy: 0.887
0.05826771653543307
0.6981132075471698
0.10755813953488373
train accuracy: 0.889
test accuracy: 0.889
0.09763779527559055
0.6813186813186813
0.17079889807162532
train accuracy: 0.888
test accuracy: 0.890
0.08346456692913386
0.7681159420289855
0.15056818181818182
train accuracy: 0.890
test accuracy: 0.889
0.07886435331230283
0.7352941176470589
0.14245014245014245


In [ ]:
#Grisserch: ExtraTreesClassifier

# params = {'n_estimators'  : [10, 100, 1000, 10000], 
#          }
# gs_ex = GridSearchCV(ExtraTreesClassifier(), params, cv = kf_1)
# gs_ex.fit(X_train, y_train)
# print(gs_ex.score(X_train,y_train))

In [19]:
#ExtraTreesClassifierの学習
print("ext")
ext_train, ext_test,ext_train_pro, ext_test_pro=learn_model(ext)

ext
train accuracy: 0.894
test accuracy: 0.895
0.14330708661417324
0.7777777777777778
0.2420212765957447
train accuracy: 0.895
test accuracy: 0.892
0.14015748031496064
0.7007874015748031
0.23359580052493437
train accuracy: 0.894
test accuracy: 0.893
0.14803149606299212
0.6962962962962963
0.24415584415584415
train accuracy: 0.894
test accuracy: 0.895
0.14645669291338584
0.7560975609756098
0.2453825857519789
train accuracy: 0.895
test accuracy: 0.890
0.10725552050473186
0.6868686868686869
0.18553888130968624


In [1]:
#GridSearch :SVC

# param_grid = {'C': [0.1, 1.0, 10],
#  'gamma': [0.001, 0.01]}
# gs_svc = GridSearchCV(SVC(), param_grid, cv=kf_1)
# gs_svc.fit(X_train,y_train)

In [ ]:
# gs_svc.best_params_

In [2]:
#SVCの学習（精度が悪く、学習時間が長いため使用しない）
# print("svc")
# svm_train, svm_test,svm_train_pro, svm_test_pro=learn_model(rbf_svm)

In [20]:
#lightGBMの学習
print("lgb")
lgb_train, lgb_test,lgb_train_pro, lgb_test_pro=learn_model(lgb)

lgb


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


train accuracy: 1.000


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


test accuracy: 0.901


...

C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.636986301369863


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5205223880597014


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
#XGBoostの学習
print("xgb")
xgb_train, xgb_test,xgb_train_pro, xgb_test_pro=learn_model(xgb)

xgb


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


train accuracy: 0.996


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


test accuracy: 0.904


...

C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6232227488151659


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.49810606060606066


C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuri\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


ロジスティック回帰追加

In [22]:
from sklearn.preprocessing import StandardScaler
#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_std_ba = scaler.transform(X_train)
X_std_test=scaler.transform(X_test)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)


In [25]:
#引数にtrainデータとtestデータを指定する以外上のモデルと一緒（標準化処理を加えたから）
def learn_log(model,X_train,X_test):
    kf =StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    ntrain=X_train.shape[0]
    ntest=X_test.shape[0]
    mo_train=np.zeros((ntrain,))
    mo_test = np.zeros((ntest,))
    mo_test_m = np.empty((5, ntest))
    mo_train_pro=np.zeros((ntrain,))
    mo_test_pro = np.zeros((ntest,))
    mo_test_m_pro = np.empty((5, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(X_train,y_train)):
        X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
        y_train_1, y_test_1 = y_train[train_index], y_train[test_index]

        model.fit(X_train_1,y_train_1)
        print('train accuracy: %.3f' % model.score(X_train_1, y_train_1))
        print('test accuracy: %.3f' % model.score(X_test_1, y_test_1))
        # 再現率を出力
        print(metrics.recall_score(y_test_1, model.predict(X_test_1)))
        # 適合率を出力
        print(metrics.precision_score(y_test_1, model.predict(X_test_1)))
        # F値を出力
        print(metrics.f1_score(y_test_1, model.predict(X_test_1)))

        mo_train[test_index] =model.predict(X_test_1)
        mo_test_m_pro[i, :] = model.predict(X_test)
        mo_train_pro[test_index] =model.predict_proba(X_test_1)[:,1]
        mo_test_m_pro[i, :] = model.predict_proba(X_test)[:,1]
    mo_test[:]=mo_test_m.mean(axis=0)
    mo_test_pro[:]=mo_test_m_pro.mean(axis=0)
    # print(mo_test.reshape(-1,1))
    return mo_train.reshape(-1, 1), mo_test.reshape(-1, 1),mo_train_pro.reshape(-1, 1), mo_test_pro.reshape(-1, 1)

In [26]:
log_train, log_test, log_train_pro, log_test_pro=learn_log(lr,X_std_ba,X_std_test)

train accuracy: 0.902
test accuracy: 0.901
0.33385826771653543
0.6483180428134556
0.4407484407484408
train accuracy: 0.903
test accuracy: 0.895
0.3165354330708661
0.5946745562130178
0.4131551901336074
train accuracy: 0.901
test accuracy: 0.902
0.3590551181102362
0.6514285714285715
0.4629441624365483
train accuracy: 0.900
test accuracy: 0.904
0.36850393700787404
0.6610169491525424
0.47320525783619827
train accuracy: 0.901
test accuracy: 0.904
0.3359621451104101
0.6805111821086262
0.44984160506863785


1回目の学習を行ったデータが消えてしまうとまた実行し直すのが大変だからcsvファイルにしておく

In [23]:
#trainデータの1回目の学習で予測したラベル0か1
#今回は基本的にラベルではなく確率を使用

# base_pre_train = pd.DataFrame( {'RandomForest': rfc_train.ravel(),
#      'ExtraTrees': ext_train.ravel(),
#      'AdaBoost': ada_train.ravel(),
# #         'svm': svm_train.ravel(),
#       'GradientBoost': gbct_train.ravel(),
#          "Logistic":log_train.ravel(),
#         "lightGBM":lgb_train.ravel(),
#         "XGBoost":xgb_train.ravel()
                            
                                
#     })
# print('base_pre_train.shape : ', base_pre_train.shape)
# base_pre_train.head(5)

base_pre_train.shape :  (18988, 7)


,RandomForest,ExtraTrees,AdaBoost,GradientBoost,Logistic,lightGBM,XGBoost
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#1回目の学習でtrainデータのラベルを予測した時の口座開設確率
base_pre_train_pro = pd.DataFrame( {'RandomForest': rfc_train_pro.ravel(),
     'ExtraTrees': ext_train_pro.ravel(),
     'AdaBoost': ada_train_pro.ravel(),
#         'svm': svm_train.ravel(),
      'GradientBoost': gbct_train_pro.ravel(),
         "Logistic":log_train_pro.ravel(),
        "lightGBM":lgb_train_pro.ravel(),
        "XGBoost":xgb_train_pro.ravel()
                            
                                
    })
print('base_pre_train_pro.shape : ', base_pre_train_pro.shape)
base_pre_train_pro.head(5)

base_pre_train_pro.shape :  (27126, 7)


,RandomForest,ExtraTrees,AdaBoost,GradientBoost,Logistic,lightGBM,XGBoost
0,0.624752,0.141266,0.465792,0.999319,0.421830,0.984461,0.979672
1,0.267777,0.108078,0.386848,0.402412,0.023375,0.002371,0.231638
2,0.650472,0.414676,0.464963,0.990537,0.910236,0.999641,0.959982
3,0.535363,0.308564,0.444146,0.879123,0.502835,0.376527,0.839066
4,0.726159,0.609523,0.524346,0.999958,0.925477,0.999928,0.992786


In [28]:
#1回目の学習でtestデータのラベルを予測した時の口座開設確率
base_pre_test_pro = pd.DataFrame( {'RandomForest': rfc_test_pro.ravel(),
     'ExtraTrees': ext_test_pro.ravel(),
     'AdaBoost': ada_test_pro.ravel(),
#         'svm': svm_train_pro.ravel(),
      'GradientBoost': gbct_test_pro.ravel(),
      "Logistic":log_test_pro.ravel(),
        "lightGBM":lgb_test_pro.ravel(),
        "XGBoost":xgb_test_pro.ravel()
                                    
    })
print('base_pre_train.shape : ', base_pre_test_pro.shape)
base_pre_test_pro.head(5)

base_pre_train.shape :  (18083, 7)


,RandomForest,ExtraTrees,AdaBoost,GradientBoost,Logistic,lightGBM,XGBoost
0,0.591686,0.238553,0.493959,0.987571,0.853369,9.684065e-01,0.982587
1,0.669127,0.136977,0.457117,0.859219,0.483809,8.662948e-01,0.958397
2,0.343893,0.039500,0.265986,0.173845,0.000062,1.510471e-07,0.010654
3,0.336631,0.042076,0.286329,0.238146,0.003138,7.884098e-08,0.034408
4,0.273175,0.128309,0.432357,0.028788,0.186114,2.228997e-02,0.731095


In [5]:
#testデータの1回目の学習で予測したラベル0か1

# base_pre_test = pd.DataFrame( {'RandomForest': rfc_test.ravel(),
#      'ExtraTrees': ext_test.ravel(),
#      'AdaBoost': ada_test.ravel(),
# #         'svm': svm_train_pro.ravel(),
#       'GradientBoost': gbct_test.ravel(),
#       "Logistic":log_test.ravel(),
#         "lightGBM":lgb_test.ravel(),
#         "XGBoost":xgb_test.ravel()
                                    
#     })
# print('base_pre_test.shape : ', base_pre_test.shape)
# base_pre_test.head(5)

以下で保存

In [ ]:
base_pre_train_pro.to_csv("base_pre_train_pro2.csv")

In [ ]:
base_pre_train.to_csv("base_pre_train_pro2.csv")

In [ ]:
base_pre_test.to_csv("base_pre_test2.csv")

In [ ]:
base_pre_test_pro.to_csv("base_pre_test_pro2.csv")

2回目の学習を行うためのデータの作成

In [31]:
x_train = np.concatenate((  rfc_train_pro,  gbct_train_pro,log_train_pro
#                           svc_train_pro
                          ,ada_train_pro,ext_train_pro,log_train_pro,lgb_train_pro,xgb_train_pro), axis=1)
x_test = np.concatenate((  rfc_test_pro,  gbct_test_pro, log_test_pro
#                           svc_test_pro
                         ,ada_test_pro,ext_test_pro,log_test_pro, lgb_test_pro,xgb_test_pro
                        ), axis=1)
print('x_train.shape : ', x_train.shape)
print('x_test.shape : ', x_test.shape)

x_train.shape :  (27126, 8)
x_test.shape :  (18083, 8)


In [ ]:
#こちらは０、1で予測したものを学習させるときのデータ

# x_train1 = np.concatenate((  rfc_train,  gbct_train,log_train
#                           svc_train,ada_train,ext_trainlog_train, lgb_train,xgb_train
#                          ), axis=1)
# x_test1 = np.concatenate((  rfc_test,  gbct_test,log_test 
#                           svc_test,ada_test,ext_test,log_test, lgb_test,xgb_test
#                         ), axis=1)
# print('x_train.shape : ', x_train1.shape)
# print('x_test.shape : ', x_test1.shape)

2回目の学習RandomForest

In [32]:
rfc_3=RandomForestClassifier(random_state=0, n_estimators=500)
rfc_3.fit(x_train, y_train)
predictions = rfc_3.predict(x_test)
predictions_pro = rfc_3.predict_proba(x_test)

In [ ]:
# rfc_3=RandomForestClassifier(random_state=0, n_estimators=500)
# rfc_3.fit(x_train1, y_train)
# predictions1 = rfc_3.predict(x_test1)

テスト用のみ


In [ ]:
rfc_3.score(x_test,y_test)

In [ ]:
rfc_3.score(x_train1,y_train)

ロジスティック

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
X_std = scaler.transform(x_train)
x_test_std=scaler.transform(x_test)


In [34]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)
lr.fit(X_std, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

テスト用のみ

In [ ]:
# 正答率を出力
print('train acc: %.3f' % lr.score(X_std, y_train))
print('test acc: %.3f' % lr.score(x_test_std, y_test))


XGBoost

In [35]:
import xgboost as xgb
xgb = xgb.XGBClassifier(learning_rate=0.1,max_depth=10)
xgb.fit(x_train,y_train)
pre_xgb = xgb.predict_proba(x_test)


テスト用のみ

In [ ]:
# 正答率を出力
print('train acc: %.3f' % xgb.score(x_train, y_train))
print('test acc: %.3f' % xgb.score(x_test, y_test))


csvファイルにして保存

In [ ]:
predic_df=pd.DataFrame(predictions_pro)
predic_df.to_csv("result_rf_new2_pro_log3.csv")
predic_log_pro=lr.predict_proba(x_test_std)
predic_log_df=pd.DataFrame(predic_log_pro)
pre_xgb=pd.DataFrame(pre_xgb)
predic_log_df.to_csv("result_log_new2_pro_log3.csv")
pre_xgb.to_csv("result_xgb_pro.csv")